### Hybrid Search Langchain

In [3]:
%pip install --upgrade --quiet  pinecone pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [ ]:
api_key=""

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

## Initialize the Pinecone Client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimension of dense vector
        metric='dotproduct', ## sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws', region="us-east-1"),
    )

In [7]:
index=pc.Index(index_name)
index

In [ ]:
## vector embeddings Dense Matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HUGGINGFACE_API_KEY_Read")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
## vector embeddings Dense Matrix
from pinecone_text.sparse import BM25Encoder ##Uses TFIDF technique

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package stopwords to C:\Users\Suyash
[nltk_data]     Mohanty\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [12]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

##tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [14]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000024CCBEE0790>, index=<pinecone.db_data.index.Index object at 0x0000024C8DB37F70>)

In [15]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
retriever.invoke("What city did I visit last")

[Document(metadata={'score': 0.271121562}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.255950719}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.235930055}, page_content='In 2023, I visited Paris')]